In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17559411
paper_name = 'shuster_rosenberg_2007' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [5]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [22]:
original_data = pd.read_excel('raw_data/femsyr_268_Table_S1.xlsx', sheet_name='Table 1', skiprows=1)

In [23]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 28 x 5


In [24]:
original_data.head()

,Unnamed: 0,MACH negative,Reduced,Slow growers,Unnamed: 4
0,Alcohol\nmetabolism,"ALD4, ACS1","ADH1, ADH2, ALD6",NaN,NaN
1,Lipid metabolism,"CHO2, HFA1, ISC1, MCT1, UPC2","AGP2, DEP1, ERG6,\nERG28, GPT2, OAR1, PSD1, SA...","CEM1,HTD2, LCB5, PDX3",NaN
2,Carbohydrate metabolism,"FUM1, MLS1",NaN,"SNF1, ACO1, KGD2",NaN
3,Amino acid\nmetabolism,ARG82,"HIS7, ILV1, AAT1,\nAAT2","CAR2, CYS3, CYS4, PUT2",NaN
4,Ubiquinone biosynthesis and\nmetabolism,NaN,NaN,"CAT5, COQ2, COQ3, COQ4, COQ5",NaN


In [25]:
for c in ['MACH negative','Reduced','Slow growers']:
    
    original_data[c] = original_data[c].apply(lambda x: [y.split('(')[0].strip().strip('\n') for y in str(x).split(',')])

In [26]:
orf1 = [orf for list_ in original_data['MACH negative'] for orf in list_]

In [28]:
orf2 = [orf for list_ in original_data['Reduced'] for orf in list_]

In [30]:
orf3 = [orf for list_ in original_data['Slow growers'] for orf in list_]

In [31]:
orfs = list(set(orf1+orf2+orf3))

In [37]:
original_data = pd.DataFrame(index=orfs, columns=['mach','growth'], data=0)

In [39]:
original_data.loc[orf1,'mach'] = -2
original_data.loc[orf1,'growth'] = 1
original_data.loc[orf2,'mach'] = -1
original_data.loc[orf2,'growth'] = 1
original_data.loc[orf3,'growth'] = -1

In [41]:
original_data = original_data.reset_index()

In [42]:
original_data.head()

,index,mach,growth
0,CYC1,-2,1
1,,-1,-1
2,TIF3,-2,1
3,RPB4,-1,1
4,PET117,0,-1


In [44]:
original_data['genename'] = original_data['index'].astype(str)

In [45]:
# Eliminate all white spaces & capitalize
original_data['genename'] = clean_genename(original_data['genename'])

In [46]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genename'], to='orf')

In [47]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            index  mach  growth  orf genename
index_input                                  
1                    -1      -1              
388           nan    -1      -1  NAN      NAN


In [48]:
original_data = original_data.loc[t,:]

In [49]:
original_data.set_index('orf', inplace=True)

In [50]:
original_data = original_data[['mach','growth']].copy()

In [51]:
original_data = original_data.groupby(original_data.index).mean()

In [52]:
original_data.shape

(461, 2)

In [53]:
original_data.head()

,mach,growth
orf,,
YAL009W,-2,1
YAL010C,-2,1
YAL012W,0,-1
YAL013W,-1,1
YAL014C,-2,1


# Prepare the final dataset

In [55]:
data = original_data.copy()

In [56]:
dataset_ids = [16705, 21881]
datasets = datasets.reindex(index=dataset_ids)

In [57]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [58]:
data.head()

dataset_id,16705,21881
data_type,value,value
orf,,
YAL009W,-2,1
YAL010C,-2,1
YAL012W,0,-1
YAL013W,-1,1
YAL014C,-2,1


## Subset to the genes currently in SGD

In [59]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [60]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16705,21881
,data_type,value,value
gene_id,orf,,
7,YAL009W,-2,1
8,YAL010C,-2,1
10,YAL012W,0,-1
11,YAL013W,-1,1
12,YAL014C,-2,1


# Normalize

In [61]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [62]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [63]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16705,21881,16705,21881
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
7,YAL009W,-2,1,-5.388842,3.183590
8,YAL010C,-2,1,-5.388842,3.183590
10,YAL012W,0,-1,0.000000,-3.119918
11,YAL013W,-1,1,-2.694421,3.183590
12,YAL014C,-2,1,-5.388842,3.183590


# Print out

In [64]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [65]:
from IO.save_data_to_db3 import *

In [66]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 17559411...
Inserting the new data...


100%|██████████| 2/2 [00:01<00:00,  1.14it/s]

Updating the data_modified_on field...
